> **Jupyter slideshow:** This notebook can be displayed as slides. To view it as a slideshow in your browser type in the console:


> `> jupyter nbconvert [this_notebook.ipynb] --to slides --post serve`


> To toggle off the slideshow cell formatting, click the `CellToolbar` button, then `View --> Cell Toolbar --> None`

<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Introduction to Support Vector Machines (SVM)

_Authors: Kiefer Katovich (SF)_

---

### Learning Objectives
- Understand how the SVM builds its decision threshold.
- Understand the concept of the maximum margin hyperplane.
- Visualize the linearly separable case in classification.
- Understand the math behind finding the maximum margin hyperplane.
- Understand the hinge loss for SVM.
- Understand how the regularization constant C allows SVMs to fit non-linearly separable problems.
- See how the kernel trick transforms problems from non-linearly separable to linearly separable.

### Lesson Guide
- [Introduction to SVMs](#intro)
- [How does the SVM classifiy?](#how-classify)
- [Intuition behind the decision boundary](#intuition)
- [Why maximize the margin?](#why-max-margin)
- [SVM origins: the perceptron algorithm](#perceptron)
- [The maximum margin hyperplane](#mmh)
    - [Finding the maximum margin](#finding-mmh)
- [The hinge loss](#hinge)
    - ["Slack"](#slack)
    - [The regularizing hyperparameter C](#hyper-c)
- [The kernel trick](#kernel)
- [Pros and cons](#pros-and-cons)
- [When to use SVM vs. Logistic Regression](#when-to-use-svm)
- [Additional resources](#resources)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC

from ipywidgets import interact
from IPython.display import display

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

<a id='intro'></a>

## Introduction to SVMs

---

The Support Vector Machine (SVM) algorithm is a different approach to classification.

SVM still fits a decision boundary like a logistic regression regression, but uses a different loss function called the "hinge loss" (as opposed to the log loss in logistic regression).

This lesson will overview the details of the SVM algorithm.

<a id='how-classify'></a>

## How does the SVM classify?

---

It's important to start with the intuition for SVM with the special _**linearly separable**_ classification case.

If classification of observations is "linearly separable", SVM fits the **"decision boundary"** that is defined by the largest margin between the closest points for each class. This is commonly called the **"maximum margin hyperplane (MMH)"**.

![linearly separable SVM](./assets/linear_separability_vs_not.png)

<a id='intuition'></a>

## Intuition behind the SVM decision boundary

---

SVM's criterion for a decision surface is one that is _maximally far away from any data point between classes_. The distance from the decision boundary to the closest data point determines the "margin" of the classifier.

The points SVM uses to fit the decision boundary are called "support vectors".

![linearly separable SVM MMH margin](./assets/Margin.png)

<a id='why-max-margin'></a>

## Why maximize the margin?

---

**SVM solves for a decision boundary that should theoretically minimize the generalization error.** 

Observations that are near the decision boundary between the classes are the observations with the most "ambiguous" labels. They are the observations that are approaching equal probability to be one class or the other (given the predictors).

SVM, instead of considering all the observations "equally" in the loss function it minimizes, defines it's fit using the most ambiguous points. It's decision boundary is safe in that errors in new measured observations are not likely to cause the SVM to mis-classify.

The SVM is concerned with generalization to new data.

<a id='perceptron'></a>

## SVM origins: the perceptron algorithm

---

The perceptron algorithm is a linear function of weights $w$ and predictors $X$ that assigns points to binary classes. If the function returns a value greater than 0 then the observation is classified as 1, otherwise it is classified as zero.

$f(X)$ below is the perceptron function to determine the classes. $b$ here is known as the "bias", which is analagous to the intercept term.

### $$ f(X) = \begin{cases}1 & \text{if }w \cdot x + b > 0\\0 & \text{otherwise}\end{cases} $$

If the points are linearly seperable, the solving the perceptron is guaranteed to converge on a solution, but that solution is not necessarily optimal for future observations. This led to the invention of the SVM, which finds the optimal discriminator: the maximum margin hyperplane.



<a id='mmh'></a>

## The maximum margin hyperplane

---

We choose a normalizing constant such that the distance from the plane to the closest points of either classes will be 1.

### $$ w^T x_{+} + b = 1 \\ w^T x_{-} + b = -1 $$

For the distance to the closest positive and negative class points, respectively (known as "support vectors").

If the normalizer for the weights is $||w||$, the size of the margin is then:

### $$ \text{margin} = \frac{2}{||w||} $$


![linearly separable SVM MMH margin](./assets/linear_sep_support_vecs_math.png)

<a id='finding-mmh'></a>
### Finding the maximum margin

We want to find a distance between these points that is the widest possible. Therefore, we are looking to maximize the value $\frac{2}{||w||}$.

So, maximize the weights with constraints on what the function can output. When the target is 1, the function needs to be 1 or greater. When the target is 0 (or -1), the function needs to output -1 or lower.

### $$ \underset{w}{\text{max}} \frac{2}{||w||} \quad \text{subject to} \begin{cases}\text{if } y_i = 1 & w^T x_i + b \ge 1 \\ \text{if } y_i = -1 & w^T x_i + b \le -1 \end{cases} \text{for } i \text{ in } N$$

Note here that $y$ is specified as either 1 or -1, as opposed to the 0, 1 that we are used to. This is convenient for converting this to be a minimization. To make this a minimization, we can change the equation to be:

### $$ \underset{w}{\text{min }} ||w||^2 \quad \text{subject to} \quad y(w^T x_i + b) \ge 1 $$

Which works out because when $y = -1$ the values become positive.

![linearly separable SVM MMH margin](./assets/linear_sep_support_vecs.png)

<a id='hinge'></a>

## The hinge loss and non-linearly separable cases

---

In cases where there is no line or plane that can separate all of the points perfectly, we need to introduce the capacity for model error. Using the constraint above that $y(w^T X + b) \ge 1$, we can introduce the **hinge loss function**:

### $$ \text{hinge loss} = \sum_{i=1}^n \max\left(0, 1 - y_i(w^T x_i + b)\right) $$

Where now, if the point is on the correct side (correctly classified), the value will be 0. If the point is not $\ge 1$, this function will be greater than zero.

Using $f(x_i) = (w^T x_i + b)$, 

### $$\text{hinge loss} = \sum_{i=1}^N max\left(0, 1 - y_i \: f(x_i)\right)$$ 

If $f(x_i) > 1$, the point lies _outside_ the margin and does not contribute to the loss.

If $f(x_i) = 1$ the point is _on_ the margin and does not contribute to the loss.

If $f(x_i) < 1$ the point lies _inside_ the margin and **does** contribute to the loss.


![Hinge loss](./assets/hinge_loss.png)

<a id='slack'></a>

### Hinge loss and "slack"

When we have a scenario where it is not possible to perfectly separate, we use the hinge loss with a regularization constant $C$:

### $$ \min ||w||^2 + C\sum_{i=1}^N \epsilon_i \quad \text{subject to} \quad y(w^T x_i + b) \ge 1 - \epsilon_i $$

Where the $\epsilon$ are the errors of the classifier, and the $C$ is a regularization term that determines how much the classification errors matter (relative to the maximization of the margin).

The function that the SVM minimizes to find the boundary is:

### $$  \underset{w}{\text{min }} ||w||^2 + C\sum_{i=1}^N max\left(0, 1 - y_i(w^T x_i + b)\right) $$

A small $C$ creates a wider margin because errors will matter less. A large $C$ creates a tighter margin because errors matter more. An infinite $C$ parameter is a "hard" margin, which always minimizes error over the size of the boundary.

We are trying to minimize the norm of the weights $||w||$ and thus maximize the margin, but now we are also trying to minimize our errors at the same time. We have a balance in our minimization between how wide the margin should be and how much error we tolerate.


<a id='hyper-c'></a>
### The regularizing hyper-parameter $C$

By setting the hyper-parameter $C$ we can control the extent to which our SVM is tolerant to misclassification errors. It is sometimes called the "soft-margin constant". 

$C$ affects the strength of the "penalty", similar to the lambda terms in the Ridge and Lasso. 

By multiplying the sum of the errors, which are the distances from the margins to the points inside of the margin, it allows the SVM to classify non-linearly separable problems by allowing errors to occur. 

The lower the value of $C$, the more misclassified observations errors are allowed. These misclassified points are known as "slack variables". Reducing the effect of errors on the loss function puts the emphasis on widening the margin.

For those interested in exporing the math more, [there is a good tutorial here.](http://www.svm-tutorial.com/2015/06/svm-understanding-math-part-3/#more-457)

![soft margin](./assets/slack_variables.png)

In [ ]:
from sklearn.datasets.samples_generator import make_blobs

def plot_svc_decision_function(clf, ax=None):
    """Plot the decision function for a 2D SVC"""
    if ax is None:
        ax = plt.gca()
    x = np.linspace(plt.xlim()[0], plt.xlim()[1], 30)
    y = np.linspace(plt.ylim()[0], plt.ylim()[1], 30)
    Y, X = np.meshgrid(y, x)
    P = np.zeros_like(X)
    for i, xi in enumerate(x):
        for j, yj in enumerate(y):
            P[i, j] = clf.decision_function(np.array([xi, yj]).reshape(1, -1))
    # plot the margins
    ax.contour(X, Y, P, colors='k',
               levels=[-1, 0, 1], alpha=0.5,
               linestyles=['--', '-', '--'])

plt.figure(figsize=(10,6))
ax = plt.gca()
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel('X1', fontsize=14)
ax.set_ylabel('X2', fontsize=14)

X, y = make_blobs(n_samples=50, centers=2,
                  random_state=121, cluster_std=0.60)

plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap=plt.cm.flag, edgecolors='black', linewidth=1);

![soft margin](./assets/soft_margin.png)

In [ ]:
def plot_linear_svc(C):
    clf = SVC(kernel='linear', C=C)
    clf.fit(X, y)
    plt.title('Linear, C={}'.format(C))
    ax = plt.gca()
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel('X1', fontsize=14)
    ax.set_ylabel('X2', fontsize=14)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap=plt.cm.flag, edgecolors='black', linewidth=1)
    plot_svc_decision_function(clf);
    
def plot_interactive_linear_svc(X, y):
    def interactive_linear_svc(C1, C2):
        plt.figure(figsize=(12,6))
        plt.subplot(121)
        plot_linear_svc(C=C1)
        plt.subplot(122)
        plot_linear_svc(C=C2)
        plt.show()
        plt.close()
    
    interact(interactive_linear_svc, C1=(.1, 1),  C2=(1, 10))
    
plot_interactive_linear_svc(X, y)

<a id='kernel'></a>
## The "kernel trick" for non-linearly separable problems

---

The "kernel trick" allows an SVM to classify non-linearly separable problems. It is a big reason why SVMs are so popular.

The idea behind the kernel trick is that you can arbitrarily transform your observations that _have no linear separability_ by putting them into a different "dimensional space" where the DO have linear separability, fit an SVM in that higher dimensional space, and then invert the transformation of the data and the model itself back into the original space.

This is done by "wrapping" your predictors in a kernel function that transforms them into this higher dimensional space. 

[Check out these lecture slides for more detail.](http://www.robots.ox.ac.uk/~az/lectures/ml/lect3.pdf)

The following pictures should give you a general intuition for what is happening.

![kernel transform viz](./assets/kernel_trick.png)

![kernel transform viz](./assets/kernel_viz.png)

![polynomial kernel](./assets/nonlinear-1.png)

In [ ]:
def plot_poly_svc(gamma, C):
    clf = SVC(kernel='poly', degree=gamma, C=C)
    clf.fit(X, y)
    plt.title('poly={}, C={}'.format(gamma, C))
    ax = plt.gca()
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel('X1', fontsize=14)
    ax.set_ylabel('X2', fontsize=14)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap=plt.cm.flag, edgecolors='black', linewidth=1)
    plot_svc_decision_function(clf);

def plot_interactive_poly_svc(X, y):
    def interactive_poly_svc(C, poly):
        plt.figure(figsize=(12,12))
        plot_poly_svc(poly, C)
        plt.show()
        plt.close()

    interact(interactive_poly_svc, C=(.1, 10), poly=(1,5))
    
plot_interactive_poly_svc(X, y)

![gaussian kernel](./assets/nonlinear-2.png)

In [ ]:
def plot_rbf_svc(gamma, C):
    clf = SVC(kernel='rbf', gamma=gamma, C=C)
    clf.fit(X, y)
    plt.title('RBF, gamma={}, C={}'.format(gamma, C))
    ax = plt.gca()
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel('X1', fontsize=14)
    ax.set_ylabel('X2', fontsize=14)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap=plt.cm.flag, edgecolors='black', linewidth=1)
    plot_svc_decision_function(clf);
    
def plot_interactive_rbf_svc(X, y):
    def interactive_rbf_svc(C, gamma1, gamma2, gamma3, gamma4):
        plt.figure(figsize=(12,12))
        plt.subplot(221)
        plot_rbf_svc(gamma1, C)
        plt.subplot(222)
        plot_rbf_svc(gamma2, C)
        plt.subplot(223)
        plot_rbf_svc(gamma3, C)
        plt.subplot(224)
        plot_rbf_svc(gamma4, C)
        plt.show()
        plt.close()

    interact(interactive_rbf_svc, C=(.1, 10), gamma1=(.1, 1), 
                    gamma2=(1, 10), gamma3=(10, 100), gamma4=(100, 200))
    
plot_interactive_rbf_svc(X, y)

<a id="pros-and-cons"></a>
## Pros and cons
**Pros**
- Exceptional perfomance (historically widely used)
- Robust to outliers
- Effective in high dimensional data
- Can work with non-linearities
- Fast to compute even on non-linear (kernel trick)
- Low risk of overfitting

**Cons**
- Blackbox
- Can be slow on large datasets

<a id="when-to-use-svm"></a>
## When to use SVM vs. Logistic Regression
Advice from Andrew Ng:

**If there are more feature than training samples:**
    Use logistic regression or SVM without a kernel ("linear kernel")
    
**If there are about 10 times as many samples as features:**
    Use SVM with a Gaussian kernel
    
**If there are many more training samples than features:**
    Spend time feature engineering, then use logistic regression or SVM
    without a kernel

<a id='resources'></a>

## Additional resources

---

- [For a really great resource check out these slides (some of which are cannabalized in this lecture).](http://www.robots.ox.ac.uk/~az/lectures/ml/lect2.pdf)
- [This website is also a great resource, on a slightly more technical level.](http://nlp.stanford.edu/IR-book/html/htmledition/support-vector-machines-the-linearly-separable-case-1.html)
- SVM docs on [SKLearn](http://scikit-learn.org/stable/modules/svm.html)
- Iris example on [SKLearn](http://scikit-learn.org/stable/auto_examples/svm/plot_iris.html#example-svm-plot-iris-py)
- Hyperplane walkthrough on [SKLearn](http://scikit-learn.org/stable/auto_examples/svm/plot_separating_hyperplane.html#example-svm-plot-separating-hyperplane-py)
- A comprehensive [user guide](http://pyml.sourceforge.net/doc/howto.pdf) to SVM. My fav!
- A [blog post tutorial](http://www.svm-tutorial.com/2014/11/svm-understanding-math-part-2/) of understanding the linear algebra behind SVM hyperplanes. Check [part 3](http://www.svm-tutorial.com/2015/06/svm-understanding-math-part-3/) of this blog on finding the optimal hyperplane
- This [Quora discussion](https://www.quora.com/How-do-you-teach-Support-Vector-Machine-to-a-beginner-in-Machine-Learning) includes a high-level overview plus a [20min video](https://www.youtube.com/watch?v=aDbsJ_S3tIA) walking through the core "need-to-knows"
- A [slideshow introduction](http://www.cs.columbia.edu/~kathy/cs4701/documents/jason_svm_tutorial.pdf) to the optimization considerations of SVM
- A second [slideshow overview from UCF](http://www.cs.ucf.edu/courses/cap6412/fall2009/papers/Berwick2003.pdf) on the highnotes of SVM
- Andrew Ng's [notes](http://cs229.stanford.edu/notes/cs229-notes3.pdf) on SVM from CS 229
- A [FULL LECTURE](https://www.youtube.com/watch?v=eHsErlPJWUU) (1hr+) from one of my fav lecturers (Dr Yasser) on SVM. He does a followup on [kernel tricks](https://www.youtube.com/watch?v=XUj5JbQihlU) too
- A [FULL LECTURE](https://www.youtube.com/watch?v=_PwhiWxHK8o) (50min) (from MIT Opencoursewar)
- An infamous [paper](https://www.cs.cornell.edu/people/tj/publications/joachims_98a.pdf) (cited 7000+ times!) on why SVM is a great text classifier
- An [advanced discussion](http://www.icml-2011.org/papers/386_icmlpaper.pdf) of SVMs as probabilistic models